In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import os
print("import finish")

import finish


In [3]:
transform = {
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}
print("transform finish")

transform finish


In [4]:
batch_size = 32
num_workers = 2  
pin_memory = True if torch.cuda.is_available() else False 
test_dir = "/kaggle/input/testdata/test"
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.transform = transform
        self.image_names = os.listdir(test_dir) 
    
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = os.path.join(self.test_dir, image_name)
        image = Image.open(image_path).convert("RGB") 
        
        if self.transform:
            image = self.transform(image)
        
        image = image.to(device)

        return image_name, image 

test_dataset = TestDataset(test_dir=test_dir, transform=transform)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory
)

print("✅ Data is ready! test datasets are loaded.")



✅ Data is ready! test datasets are loaded.


In [15]:
import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model1 = models.resnext50_32x4d(weights=None)
model1.fc = torch.nn.Linear(model1.fc.in_features, 100)

model1.load_state_dict(torch.load("/kaggle/input/model1/pytorch/default/1/finetuned_resnext50.pth", map_location="cpu"))
model1.to(device)  
model1.eval()  

model2 = models.resnext101_32x8d(weights=None)
model2.fc = torch.nn.Linear(model2.fc.in_features, 100)

model2.load_state_dict(torch.load("/kaggle/input/model2/pytorch/default/1/finetuned_resnext50.pth", map_location="cpu"))
model2.to(device
model2.eval()  
total_params = sum(p.numel() for p in model2.parameters())
trainable_params = sum(
    p.numel() for p in model2.parameters() if p.requires_grad
)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print("✅ Models loaded and moved to GPU!")

<ipython-input-15-95b02ad9ecb2>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load("/kaggle/input/model1/pytorch/default/1/finetuned_resnext

Total parameters: 86947236
Trainable parameters: 86947236
✅ Models loaded and moved to GPU!


In [1]:
def predict_with_two_models(image_path, model1, model2, transform, topk=5):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs1 = model1(image)
        outputs2 = model2(image)

        probs1 = torch.nn.functional.softmax(outputs1, dim=1)
        probs2 = torch.nn.functional.softmax(outputs2, dim=1)

        avg_probs = (probs1 + probs2) / 2  
        
        top_probs, top_labels = torch.topk(avg_probs, topk)
    
    return top_labels.squeeze().cpu().tolist()
    
image_folder = "/kaggle/input/testdata/test"
output_csv = "./prediction.csv"

predictions = []
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    top5_labels = predict_with_two_models(image_path, model1, model2, transform['val'], topk=5)
    
    pred_label = top5_labels[0]
    predictions.append([image_name.replace('.jpg', ''), pred_label])

output_csv = "./merged_prediction.csv"
pd.DataFrame(predictions, columns=["image_name", "pred_label"]).to_csv(output_csv, index=False)

NameError: name 'os' is not defined